In [3]:
# !pip install dspy-ai
# import dspy

In [4]:
!pip install git+https://github.com/stanfordnlp/dspy.git
!pip install -U openai
import openai
!pip install datasets
!pip install joblib

from datasets import load_dataset

  Cloning https://github.com/stanfordnlp/dspy.git to /tmp/pip-req-build-uugbrzj8
  Running command git clone --filter=blob:none --quiet https://github.com/stanfordnlp/dspy.git /tmp/pip-req-build-uugbrzj8
  Resolved https://github.com/stanfordnlp/dspy.git to commit 16764d5e68d7f209814dbb54911d1583ef8f449b
  Preparing metadata (setup.py) ... done
  Using cached openai-0.28.1-py3-none-any.whl (76 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.6.1
    Uninstalling openai-1.6.1:
      Successfully uninstalled openai-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


  Using cached openai-1.6.1-py3-none-any.whl (225 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.1
    Uninstalling openai-0.28.1:
      Successfully uninstalled openai-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
dspy-ai 2.0.8 requires openai<=0.28.1, but you have openai 1.6.1 which is incompatible.


In [5]:
from google.colab import userdata
api_key = userdata.get('api_key')

In [6]:
import dspy
model=dspy.OpenAI(model='gpt-3.5-turbo', api_key=api_key)

In [7]:
dspy.settings.configure(lm=model)

In [8]:
# medqa_dataset = load_dataset("bigbio/med_qa")
# print(medqa_dataset)

In [9]:
#PubMedQA
# pubmedqa_dataset = load_dataset("bigbio/pubmed_qa")
# print(pubmedqa_dataset)

In [10]:
#MedMCQA
# medmcqa_dataset = load_dataset("medmcqa")
# print(medmcqa_dataset)

In [11]:
#MMLU: clinical knowledge, medical genetics, anatomy, professional medicine, college biology, and college medicine. These are the datasets we will be using for our project.
# knowledges = ['clinical_knowledge','medical_genetics','anatomy', 'professional_medicine', 'college_biology', 'college_medicine']
# mmlu_dataset_professional = load_dataset("cais/mmlu",'professional_medicine')

In [12]:
medqa_dataset = load_dataset("bigbio/med_qa")

#training on the first 20 questions
train_subset = medqa_dataset["train"][:20]

train_questions = train_subset["question"]
train_answers_id = train_subset["answer_idx"]
train_options= train_subset["options"]
train_answer = train_subset["answer"]

In [13]:
val_subset = medqa_dataset["validation"][:20]

val_questions = val_subset["question"]
val_answers_id = val_subset["answer_idx"]
val_options= val_subset["options"]
val_answer = val_subset["answer"]

formatted_val_options = [' '.join(f"{option['key']} {option['value']}" for option in options) for options in val_options]

In [14]:
print(medqa_dataset)

DatasetDict({
    train: Dataset({
        features: ['meta_info', 'question', 'answer_idx', 'answer', 'options'],
        num_rows: 10178
    })
    test: Dataset({
        features: ['meta_info', 'question', 'answer_idx', 'answer', 'options'],
        num_rows: 1273
    })
    validation: Dataset({
        features: ['meta_info', 'question', 'answer_idx', 'answer', 'options'],
        num_rows: 1272
    })
})


In [15]:
# # Data processing
formatted_train_options = [' '.join(f"{option['key']} {option['value']}" for option in options) for options in train_options]
# print(formatted_train_options)

In [16]:
# We are expecting a single choice answer so signature accordingly.
class MultipleChoiceQA(dspy.Signature):
    """Answer questions with single letter answers."""

    question = dspy.InputField(desc="The multiple-choice question.")
    options = dspy.InputField(desc="The set of options in the format : A option1 B option2 C option3 D option4 E option5 where A corresponds to option1, B to option2 and so on.")
    answer = dspy.OutputField(desc="A single-letter answer corresponding to the selected option.")

In [17]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [18]:
print(train_questions[1])
print(train_options[1])
print(train_answer[1])
print(train_answers_id)

A 3-month-old baby died suddenly at night while asleep. His mother noticed that he had died only after she awoke in the morning. No cause of death was determined based on the autopsy. Which of the following precautions could have prevented the death of the baby?
[{'key': 'A', 'value': 'Placing the infant in a supine position on a firm mattress while sleeping'}, {'key': 'B', 'value': 'Routine postnatal electrocardiogram (ECG)'}, {'key': 'C', 'value': 'Keeping the infant covered and maintaining a high room temperature'}, {'key': 'D', 'value': 'Application of a device to maintain the sleeping position'}, {'key': 'E', 'value': 'Avoiding pacifier use during sleep'}]
Placing the infant in a supine position on a firm mattress while sleeping
['E', 'A', 'A', 'A', 'E', 'C', 'E', 'A', 'E', 'E', 'C', 'D', 'D', 'C', 'D', 'C', 'E', 'D', 'E', 'B']


In [19]:
# generate_answer = dspy.Predict(MultipleChoiceQA)
# dev_example=train_questions[0]
# pred = generate_answer(question=dev_example,options=formatted_train_options[0])
# print(f"Question: {dev_example}")
# print(f"Predicted Answer: {pred.answer}")

In [20]:
# generate_answer = dspy.Predict(BasicQA)
# dev_example='What is the capital of India?'
# pred = generate_answer(question=dev_example)
# print(f"Question: {dev_example}")
# print(f"Predicted Answer: {pred.answer}")

In [21]:
# Shuffling the questions.
import random
generate_answer = dspy.Predict(MultipleChoiceQA)

def generate_responses(questions: list[str], option_sets: list[str]) -> list[str]:
    responses = []
    combined_data = list(zip(questions, option_sets))
    random.shuffle(combined_data)
    shuffled_questions, shuffled_option_sets = zip(*combined_data)

    for question, options in zip(shuffled_questions, shuffled_option_sets):
        pred_response = generate_answer(question=question, options=options)
        generated_response = pred_response.answer
        responses.append(generated_response)

    return responses

# def generate_responses(questions, option_sets):
#     responses = []
#     for question, options_list in zip(questions, option_sets):
#         pred_response = generate_answer(question=question, options=options_list)
#         generated_response = pred_response.answer
#         responses.append(generated_response)
#     return responses

In [22]:
def test_generate_responses():
  assert generate_responses(['What are the symptoms of a headache?'],['A fever B cold C cough D high bp E migraine']) == ['E']
test_generate_responses()

In [23]:
def calculate_medqa_accuracy(ques, pred: list[str], true: list[str])-> float:
    correct_count = 0

    for p, t in zip(pred, true):
        if p == t:
            correct_count += 1

    accuracy = correct_count / len(pred)
    return accuracy

In [24]:
def validate_answer(ques, pred: list[str], ref: list[str])-> bool:
  return [p==t for p, t in zip(pred, ref)]

In [25]:
def create_train_set(train_questions, train_options, train_answers):
    train_set = []

    for question, options, answer in zip(train_questions, train_options, train_answers):
        example = dspy.Example(
            question=question,
            options=options,
            answer=answer
        ).with_inputs("question", "options")

        train_set.append(example)

    return train_set

In [26]:
val_set = create_train_set(val_questions, formatted_val_options, val_answers_id)

In [27]:
print(len(val_set))

20


In [28]:
train_set3 = create_train_set(train_questions, formatted_train_options, train_answers_id)

In [29]:
print(len(train_set3))

20


In [30]:
print(train_questions)
print(formatted_train_options)
print(train_answers_id)

['A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?', 'A 3-month-old baby died suddenly at night while asleep. His mother noticed that he had died only after she awoke in the morning. No cause of death was determined based on the autopsy. Which of the following precautions could have prevented the death of the baby?', "A mother brings her 3-week-old infant to the pediatrician's office because she is concerned about his feeding habits. He was born without complic

In [31]:
train_set = [dspy.Example(question="Which of the following is the primary function of insulin?", options="A Glycogen breakdown B Protein synthesis C Glucose uptake D Lipid digestion E Oxygen transport", answer="C").with_inputs("question","options"),
dspy.Example(question="What is the most common cause of acute pancreatitis?", options="A Alcohol consumption B Gallstones C Trauma D Viral infection E Genetic predisposition", answer="B").with_inputs("question","options"),
dspy.Example(question="Which of the following is a characteristic finding in rheumatoid arthritis?", options="A HLA-B27 positivity B Anti-dsDNA antibodies C Rheumatoid factor positivity D Anti-Smith antibodies E Decreased C-reactive protein", answer="C").with_inputs("question","options"),
dspy.Example(question="In a patient with chronic renal failure, which electrolyte imbalance is commonly observed?", options="A Hyperkalemia B Hyponatremia C Hypocalcemia D Hyperphosphatemia E Hypomagnesemia", answer="A").with_inputs("question","options"),
dspy.Example(question="Which class of antibiotics is known to cause irreversible ototoxicity?", options="A Penicillins B Cephalosporins C Aminoglycosides D Tetracyclines E Macrolides", answer="C").with_inputs("question","options"),
dspy.Example(question="What is the gold standard diagnostic test for acute appendicitis?", options="A Ultrasound B CT scan C MRI D X-ray E Physical examination", answer="B").with_inputs("question","options"),
dspy.Example(question="A patient with a history of frequent sun exposure presents with a nodular, pearly lesion on the face What is the most likely diagnosis?", options="A Psoriasis B Basal cell carcinoma C Melanoma D Squamous cell carcinoma E Rosacea", answer="B").with_inputs("question","options"),
dspy.Example(question="Which hormone is responsible for stimulating milk production in the mammary glands?", options="A Progesterone B Prolactin C Estrogen D Oxytocin E Human chorionic gonadotropin (hCG)", answer="B").with_inputs("question","options"),
dspy.Example(question="What is the first-line treatment for status epilepticus?", options="A Diazepam B Phenytoin C Levetiracetam D Lorazepam E Phenobarbital", answer="D").with_inputs("question","options"),
dspy.Example(question="A patient presents with chest pain that improves with leaning forward Which condition is most likely responsible for this symptom?", options="A Myocardial infarction B Pericarditis C Angina pectoris D Aortic dissection E Pulmonary embolism", answer="B").with_inputs("question","options")]

In [32]:
train_set2 = [
    dspy.Example(question="Which of the following is the primary function of insulin?", options="A. Glycogen breakdown B. Protein synthesis C. Glucose uptake D. Lipid digestion E. Oxygen transport", answer="C").with_inputs("question","options"),
    dspy.Example(question="What is the most common cause of acute pancreatitis?", options="A. Alcohol consumption B. Gallstones C. Trauma D. Viral infection E. Genetic predisposition", answer="B").with_inputs("question","options"),
    dspy.Example(question="Which of the following is a characteristic finding in rheumatoid arthritis?", options="A. HLA-B27 positivity B. Anti-dsDNA antibodies C. Rheumatoid factor positivity D. Anti-Smith antibodies E. Decreased C-reactive protein", answer="C").with_inputs("question","options"),
    dspy.Example(question="In a patient with chronic renal failure, which electrolyte imbalance is commonly observed?", options="A. Hyperkalemia B. Hyponatremia C. Hypocalcemia D. Hyperphosphatemia E. Hypomagnesemia", answer="A").with_inputs("question","options"),
    dspy.Example(question="Which class of antibiotics is known to cause irreversible ototoxicity?", options="A. Penicillins B. Cephalosporins C. Aminoglycosides D. Tetracyclines E. Macrolides", answer="C").with_inputs("question","options"),
    dspy.Example(question="What is the gold standard diagnostic test for acute appendicitis?", options="A. Ultrasound B. CT scan C. MRI D. X-ray E. Physical examination", answer="B").with_inputs("question","options"),
    dspy.Example(question="A patient with a history of frequent sun exposure presents with a nodular, pearly lesion on the face. What is the most likely diagnosis?", options="A. Psoriasis B. Basal cell carcinoma C. Melanoma D. Squamous cell carcinoma E. Rosacea", answer="B").with_inputs("question","options"),
    dspy.Example(question="Which hormone is responsible for stimulating milk production in the mammary glands?", options="A. Progesterone B. Prolactin C. Estrogen D. Oxytocin E. Human chorionic gonadotropin (hCG)", answer="B").with_inputs("question","options"),
    dspy.Example(question="What is the first-line treatment for status epilepticus?", options="A. Diazepam B. Phenytoin C. Levetiracetam D. Lorazepam E. Phenobarbital", answer="D").with_inputs("question","options"),
    dspy.Example(question="A patient presents with chest pain that improves with leaning forward. Which condition is most likely responsible for this symptom?", options="A. Myocardial infarction B. Pericarditis C. Angina pectoris D. Aortic dissection E. Pulmonary embolism", answer="B").with_inputs("question","options"),
    dspy.Example(question="Which vitamin deficiency is associated with bleeding gums and easy bruising?", options="A. Vitamin A B. Vitamin B12 C. Vitamin C D. Vitamin D E. Vitamin K", answer="C").with_inputs("question","options"),
    dspy.Example(question="In a patient with suspected pulmonary embolism, which imaging modality is preferred for initial evaluation?", options="A. Chest X-ray B. CT pulmonary angiography C. Ventilation-perfusion scan D. Magnetic resonance imaging (MRI) E. Ultrasound", answer="B").with_inputs("question","options"),
    dspy.Example(question="A patient with a history of alcohol abuse presents with confusion, ataxia, and ophthalmoplegia. What is the most likely diagnosis?", options="A. Wernicke's encephalopathy B. Korsakoff syndrome C. Delirium tremens D. Hepatic encephalopathy E. Cerebral contusion", answer="A").with_inputs("question","options"),
    dspy.Example(question="Which of the following is a first-line treatment for hypertension in a patient with diabetes mellitus?", options="A. Atenolol B. Lisinopril C. Hydrochlorothiazide D. Diltiazem E. Clonidine", answer="B").with_inputs("question","options"),
    dspy.Example(question="What is the primary neurotransmitter involved in the pathophysiology of Alzheimer's disease?", options="A. Dopamine B. Acetylcholine C. Serotonin D. Gamma-aminobutyric acid (GABA) E. Norepinephrine", answer="B").with_inputs("question","options"),
    dspy.Example(question="A patient with a history of rheumatic heart disease presents with mitral valve stenosis. Which physical finding is characteristic of this condition?", options="A. Loud S1 heart sound B. Opening snap C. Holosystolic murmur D. Pulsus paradoxus E. Systolic ejection murmur", answer="B").with_inputs("question","options"),
    dspy.Example(question="What is the most common cause of viral myocarditis?", options="A. Coxsackievirus B. Influenza virus C. Human immunodeficiency virus (HIV) D. Herpes simplex virus E. Epstein-Barr virus", answer="A").with_inputs("question","options"),
    dspy.Example(question="Which of the following drugs is contraindicated in patients with a sulfa allergy?", options="A. Cephalexin B. Amoxicillin C. Ciprofloxacin D. Trimethoprim-sulfamethoxazole E. Doxycycline", answer="D").with_inputs("question","options"),
    dspy.Example(question="A patient presents with recurrent episodes of sudden-onset vertigo, hearing loss, and tinnitus. What is the most likely diagnosis?", options="A. Migraine-associated vertigo B. Meniere's disease C. Vestibular neuritis D. Benign paroxysmal positional vertigo (BPPV) E. Labyrinthitis", answer="B").with_inputs("question","options"),
    dspy.Example(question="Which of the following is a characteristic feature of polycystic ovary syndrome (PCOS)?", options="A. Hyperprolactinemia B. Oligomenorrhea C. Hypogonadism D. Amenorrhea E. Increased estrogen levels", answer="B").with_inputs("question","options"),
    dspy.Example(question="A patient with chronic kidney disease is prescribed a phosphate binder. Which medication is commonly used for this purpose?", options="A. Calcium carbonate B. Sevelamer C. Furosemide D. Spironolactone E. Enalapril", answer="B").with_inputs("question","options"),
    dspy.Example(question="What is the classic triad of symptoms in diabetes insipidus?", options="A. Polyuria, polydipsia, weight loss B. Polyuria, polydipsia, hyperglycemia C. Polyuria, polydipsia, hypotension D. Polyuria, polydipsia, hypernatremia E. Polyuria, polydipsia, hypoglycemia", answer="D").with_inputs("question","options"),
    dspy.Example(question="A newborn presents with abdominal distension, vomiting, and failure to pass meconium. What is the most likely diagnosis?", options="A. Intussusception B. Hirschsprung disease C. Pyloric stenosis D. Meconium ileus E. Imperforate anus", answer="D").with_inputs("question","options"),
    dspy.Example(question="Which of the following is a common side effect of angiotensin-converting enzyme (ACE) inhibitors?", options="A. Hyperkalemia B. Hypokalemia C. Hypercalcemia D. Hyponatremia E. Hypoglycemia", answer="A").with_inputs("question","options"),
    dspy.Example(question="A patient with a history of chronic alcohol abuse presents with bleeding tendencies. Which vitamin deficiency is most likely responsible for this clinical manifestation?", options="A. Vitamin A B. Vitamin B12 C. Vitamin C D. Vitamin D E. Vitamin K", answer="E").with_inputs("question","options"),
    dspy.Example(question="What is the first-line treatment for uncomplicated urinary tract infection (UTI) in a non-pregnant woman?", options="A. Ciprofloxacin B. Amoxicillin C. Trimethoprim-sulfamethoxazole D. Nitrofurantoin E. Doxycycline", answer="D").with_inputs("question","options"),
    dspy.Example(question="A patient presents with a severe headache, photophobia, and a positive Brudzinski sign. What is the most likely diagnosis?", options="A. Migraine headache B. Tension-type headache C. Cluster headache D. Subarachnoid hemorrhage E. Sinusitis", answer="D").with_inputs("question","options"),
    dspy.Example(question="Which type of hepatitis is most commonly transmitted through contaminated food and water?", options="A. Hepatitis A B. Hepatitis B C. Hepatitis C D. Hepatitis D E. Hepatitis E", answer="A").with_inputs("question","options"),
    dspy.Example(question="A patient with a history of chronic obstructive pulmonary disease (COPD) presents with increased dyspnea, wheezing, and decreased breath sounds. What is the most likely diagnosis?", options="A. Pneumonia B. Pulmonary embolism C. Pneumothorax D. Asthma exacerbation E. Pleural effusion", answer="C").with_inputs("question","options"),
    dspy.Example(question="Which medication is commonly used as a first-line treatment for acute gout attacks?", options="A. Naproxen B. Allopurinol C. Colchicine D. Methotrexate E. Prednisone", answer="C").with_inputs("question","options")
]

In [33]:
print(train_set)

[Example({'question': 'Which of the following is the primary function of insulin?', 'options': 'A Glycogen breakdown B Protein synthesis C Glucose uptake D Lipid digestion E Oxygen transport', 'answer': 'C'}) (input_keys={'options', 'question'}), Example({'question': 'What is the most common cause of acute pancreatitis?', 'options': 'A Alcohol consumption B Gallstones C Trauma D Viral infection E Genetic predisposition', 'answer': 'B'}) (input_keys={'options', 'question'}), Example({'question': 'Which of the following is a characteristic finding in rheumatoid arthritis?', 'options': 'A HLA-B27 positivity B Anti-dsDNA antibodies C Rheumatoid factor positivity D Anti-Smith antibodies E Decreased C-reactive protein', 'answer': 'C'}) (input_keys={'options', 'question'}), Example({'question': 'In a patient with chronic renal failure, which electrolyte imbalance is commonly observed?', 'options': 'A Hyperkalemia B Hyponatremia C Hypocalcemia D Hyperphosphatemia E Hypomagnesemia', 'answer': '

In [34]:
# Zero Shot
class ZERO(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.Predict(MultipleChoiceQA)

    def forward(self, question, options):
        answer = self.generate_answer(question=question,options=options)
# Using hard constrain. (Assert)
        dspy.Assert(len(answer) == 1,
        "Answer should comprise of exactly one character.")
# Using soft constraint. (Suggest)
        dspy.Suggest(len(answer) == 1,
        "Answer should comprise of exactly one character.")

        return answer


In [35]:
zero = ZERO()

In [65]:
# Chain of thought.
class COT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought(MultipleChoiceQA)

    def forward(self, question, options):
        answer = self.generate_answer(question=question,options=options)
# # Using hard constrain. (Assert)
#         dspy.Assert(len(answer) == 1,
#         "Answer should comprise of exactly one character.")
# Using soft constraint. (Suggest)
        # dspy.Suggest(len(answer) == 1,
        # "Answer should comprise of exactly one character.")

        return answer

In [66]:
cot = COT()

In [38]:
from dspy.evaluate import Evaluate
metric = dspy.evaluate.metrics.answer_exact_match

In [39]:
# Creating subset of randomly selected questions. (Random few shot)
import random
def random_few_shot_train_set(train_set, batch_size):
    return random.sample(train_set, batch_size)
batch_size = 2
random_batch = random_few_shot_train_set(train_set, batch_size)

In [40]:
# from dspy.teleprompt import KNNFewShot
# from dspy.predict.knn import KNN

# knn_teleprompter = KNNFewShot(KNN, 7, train_set)
# compiled_knn = knn_teleprompter.compile(BasicQABot(), trainset=trainset)

In [41]:
!pip install -U sentence-transformers

In [42]:
!apt-get install libomp-dev
!pip install faiss-cpu

import faiss

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libomp-dev is already the newest version (1:14.0-55~exp2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [43]:
# #knnfew shot and chain of thought
# from dspy.teleprompt import Teleprompter
# from dspy.teleprompt import BootstrapFewShot
# from dspy.predict.knn import KNN
# from sentence_transformers import SentenceTransformer
# class KNNFewShot(Teleprompter):
#     def __init__(self, KNN, k, trainset):
#         self.KNN = KNN(k, trainset)

#     def compile(self, student, *, teacher=None, trainset, valset=None):
#         student_copy = student.reset_copy()

#         def forward_pass(*args, **kwargs):
#             knn_trainset = self.KNN(**kwargs)
#             few_shot_bootstrap = BootstrapFewShot()
#             compiled_program = few_shot_bootstrap.compile(cot, trainset = knn_trainset)
#             return compiled_program(**kwargs)

#         student_copy.forward = types.MethodType(forward_pass, student_copy)
#         return student_copy

In [67]:
from dspy.teleprompt import KNNFewShot
from dspy.predict.knn import KNN

knn_teleprompter = KNNFewShot(KNN, 7, train_set)
compiled_knn = knn_teleprompter.compile(cot, trainset=train_set)

In [68]:
from dspy.evaluate.evaluate import Evaluate
evaluator = Evaluate(devset=val_set, num_threads=1, display_progress=True, display_table=0)

In [46]:
evaluator(zero, metric=metric)

Average Metric: 10 / 20  (50.0): 100%|██████████| 20/20 [00:00<00:00, 186.76it/s]

Average Metric: 10 / 20  (50.0%)


50.0

In [47]:
from dspy.teleprompt import BootstrapFewShot

teleprompter = BootstrapFewShot(metric = calculate_medqa_accuracy, max_rounds = 5)
compiled = teleprompter.compile(ZERO(), trainset=train_set)

100%|██████████| 10/10 [00:00<00:00, 200.49it/s]

Bootstrapped 0 full traces after 10 examples in round 4.


In [48]:
evaluator(compiled, metric=metric)

Average Metric: 10 / 20  (50.0): 100%|██████████| 20/20 [00:00<00:00, 122.11it/s]

Average Metric: 10 / 20  (50.0%)


50.0

In [69]:
evaluator(compiled_knn, metric=metric)




  0%|          | 0/20 [00:00<?, ?it/s]



 57%|█████▋    | 4/7 [00:00<00:00, 472.49it/s]



Average Metric: 1 / 1  (100.0):   0%|          | 0/20 [00:00<?, ?it/s]


Average Metric: 1 / 1  (100.0):   5%|▌         | 1/20 [00:00<00:02,  8.99it/s]

Bootstrapped 4 full traces after 5 examples in round 0.






 57%|█████▋    | 4/7 [00:00<00:00, 369.11it/s]



Average Metric: 2 / 2  (100.0):   5%|▌         | 1/20 [00:00<00:02,  8.99it/s]

Bootstrapped 4 full traces after 5 examples in round 0.





Average Metric: 2 / 2  (100.0):  10%|█         | 2/20 [00:00<00:02,  6.23it/s]



 57%|█████▋    | 4/7 [00:00<00:00, 388.08it/s]



Average Metric: 2 / 3  (66.7):  10%|█         | 2/20 [00:00<00:02,  6.23it/s] 


Average Metric: 2 / 3  (66.7):  15%|█▌        | 3/20 [00:00<00:02,  6.07it/s]

...



 57%|█████▋    | 4/7 [00:00<00:00, 359.27it/s]



Average Metric: 12 / 19  (63.2):  90%|█████████ | 18/20 [00:03<00:00,  4.26it/s]


Average Metric: 12 / 19  (63.2):  95%|█████████▌| 19/20 [00:03<00:00,  4.09it/s]

Bootstrapped 4 full traces after 5 examples in round 0.




 57%|█████▋    | 4/7 [00:00<00:00, 375.96it/s]



Average Metric: 12 / 20  (60.0):  95%|█████████▌| 19/20 [00:04<00:00,  4.09it/s]


Average Metric: 12 / 20  (60.0): 100%|██████████| 20/20 [00:04<00:00,  4.73it/s]

Bootstrapped 4 full traces after 5 examples in round 0.
Average Metric: 12 / 20  (60.0%)


60.0

In [70]:
import tqdm
import random

from dspy.teleprompt.teleprompt import Teleprompter

class Ensemble(Teleprompter):
    def __init__(self, *, reduce_fn=None, size=None, deterministic=False):
        """A common reduce_fn is dspy.majority."""

        assert deterministic is False, "TODO: Implement example hashing for deterministic ensemble."

        self.reduce_fn = reduce_fn
        self.size = size
        self.deterministic = deterministic

    def compile(self, programs):
        size = self.size
        reduce_fn = self.reduce_fn

        class EnsembledProgram(dspy.Module):
            def __init__(self):
                super().__init__()
                self.programs = programs

            def forward(self, *args, **kwargs):
                programs = random.sample(self.programs, size) if size else self.programs
                outputs = [prog(*args, **kwargs) for prog in programs]

                if reduce_fn:
                    return reduce_fn(outputs)

                return outputs

        return EnsembledProgram()

In [74]:
import tqdm
import random

from dspy.teleprompt.teleprompt import Teleprompter

class Ensemble(Teleprompter):
    def __init__(self, *, reduce_fn=None, size=None, deterministic=False):
        """A common reduce_fn is dspy.majority."""

        assert deterministic is False, "TODO: Implement example hashing for deterministic ensemble."

        self.reduce_fn = reduce_fn
        self.size = size
        self.deterministic = deterministic

    def compile(self, programs):
        size = self.size
        reduce_fn = self.reduce_fn

        class EnsembledProgram(dspy.Module):
            def __init__(self):
                super().__init__()
                self.programs = programs

            def forward(self, *args, **kwargs):

              for _ in range(5):
                shuffled_args = []
                for arg in args:
                    if isinstance(arg, dspy.Example):  # Check if the argument is an Example object
                        shuffled_options = random.sample(arg["options"].split(), len(arg["options"].split()))
                        shuffled_args.append(arg.with_fields(options=" ".join(shuffled_options)))
                    else:
                        shuffled_args.append(arg)
                programs = random.sample(self.programs, size) if size else self.programs
                outputs = [prog(*shuffled_args, **kwargs) for prog in programs]

                if reduce_fn:
                    return reduce_fn(outputs)

                return outputs

        return EnsembledProgram()

In [75]:
programs = [compiled_knn]
ensembled_program = Ensemble(reduce_fn=dspy.majority).compile(programs)

In [77]:
evaluator(ensembled_program, metric=metric)



  0%|          | 0/20 [00:00<?, ?it/s]


 57%|█████▋    | 4/7 [00:00<00:00, 571.06it/s]


Average Metric: 1 / 1  (100.0):   0%|          | 0/20 [00:00<?, ?it/s]

Average Metric: 1 / 1  (100.0):   5%|▌         | 1/20 [00:00<00:02,  9.25it/s]

Bootstrapped 4 full traces after 5 examples in round 0.





 57%|█████▋    | 4/7 [00:00<00:00, 957.00it/s]


Average Metric: 2 / 2  (100.0):   5%|▌         | 1/20 [00:00<00:02,  9.25it/s]

Average Metric: 2 / 2  (100.0):  10%|█         | 2/20 [00:00<00:02,  6.31it/s]

Bootstrapped 4 full traces after 5 examples in round 0.





 57%|█████▋    | 4/7 [00:00<00:00, 499.93it/s]


Average Metric: 2 / 3  (66.7):  10%|█         | 2/20 [00:00<00:02,  6.31it/s] 

Average Metric: 2 / 3  (66.7):  15%|█▌        | 3/20 [00:00<00:02,  5.82it/s]

...




 57%|█████▋    | 4/7 [00:00<00:00, 905.85it/s]


Average Metric: 11 / 19  (57.9):  90%|█████████ | 18/20 [00:03<00:00,  4.73it/s]

Average Metric: 11 / 19  (57.9):  95%|█████████▌| 19/20 [00:03<00:00,  4.42it/s]

Bootstrapped 4 full traces after 5 examples in round 0.





 57%|█████▋    | 4/7 [00:00<00:00, 372.70it/s]


Average Metric: 11 / 20  (55.0):  95%|█████████▌| 19/20 [00:04<00:00,  4.42it/s]

Average Metric: 11 / 20  (55.0): 100%|██████████| 20/20 [00:04<00:00,  4.88it/s]

Bootstrapped 4 full traces after 5 examples in round 0.
Average Metric: 11 / 20  (55.0%)


55.0

In [ ]:
model.inspect_history()

In [ ]:
# def compiled_zero(questions: list[str], option_sets: list[str]) -> list[str]:
#     responses = []
#     combined_data = list(zip(questions, option_sets))
#     random.shuffle(combined_data)
#     shuffled_questions, shuffled_option_sets = zip(*combined_data)

#     for question, options in zip(shuffled_questions, shuffled_option_sets):
#         pred_response = compiled(question=question, options=options)
#         generated_response = pred_response.answer
#         responses.append(generated_response)

#     return responses

In [ ]:
import random
def random_few_shot_train_set(train_set, batch_size):
    return random.sample(train_set, batch_size)
batch_size = 2
random_batch = random_few_shot_train_set(train_set, batch_size)
print(random_batch)